# Quick Start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/osllmai/inDox/blob/master/Demo/quick_start.ipynb)

In [ ]:
!pip install indox
!pip install openai
!pip install chromadb

## Setting Up the Python Environment

If you are running this project in your local IDE, please create a Python environment to ensure all dependencies are correctly managed. You can follow the steps below to set up a virtual environment named `indox`:

### Windows

1. **Create the virtual environment:**
```bash
python -m venv indox
```
2. **Activate the virtual environment:**
```bash
indox_judge\Scripts\activate
```

### macOS/Linux

1. **Create the virtual environment:**
   ```bash
   python3 -m venv indox
```

2. **Activate the virtual environment:**
    ```bash
   source indox/bin/activate
```
### Install Dependencies

Once the virtual environment is activated, install the required dependencies by running:

```bash
pip install -r requirements.txt
```


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

## Initial Setup

The following imports are essential for setting up the Indox application. These imports include the main Indox retrieval augmentation module, question-answering models, embeddings, and data loader splitter.

In [3]:
from indox import IndoxRetrievalAugmentation

indox = IndoxRetrievalAugmentation()

INFO: IndoxRetrievalAugmentation initialized

            ██  ███    ██  ██████   ██████  ██       ██
            ██  ████   ██  ██   ██ ██    ██   ██  ██
            ██  ██ ██  ██  ██   ██ ██    ██     ██
            ██  ██  ██ ██  ██   ██ ██    ██   ██   ██
            ██  ██  █████  ██████   ██████  ██       ██
            


### Generating response using OpenAI's language models 
OpenAIQA class is used to handle question-answering task using OpenAI's language models. This instance creates OpenAiEmbedding class to specifying embedding model. Here ChromaVectorStore handles the storage and retrieval of vector embeddings by specifying a collection name and sets up a vector store where text embeddings can be stored and queried.

In [5]:
from indox.llms import OpenAi
from indox.embeddings import OpenAiEmbedding

openai_qa = OpenAi(api_key=OPENAI_API_KEY, model="gpt-3.5-turbo-0125")
embed_openai = OpenAiEmbedding(api_key=OPENAI_API_KEY, model="text-embedding-3-small")

from indox.vector_stores import Chroma
db = Chroma(collection_name="sample",embedding_function=embed_openai)

INFO: Initializing OpenAi with model: gpt-3.5-turbo-0125
INFO: OpenAi initialized successfully
INFO: Initialized OpenAiEmbedding with model: text-embedding-3-small


### load and preprocess data
This part of code demonstrates how to load and preprocess text data from a file, split it into chunks, and store these chunks in the vector store that was set up previously.

In [4]:
!wget https://raw.githubusercontent.com/osllmai/inDox/master/Demo/sample.txt

^C


In [6]:
file_path = "sample.txt"

In [7]:
from indox.data_loader_splitter import SimpleLoadAndSplit

loader_splitter = SimpleLoadAndSplit(file_path=file_path, max_chunk_size=400)
docs = loader_splitter.load_and_chunk()

INFO: UnstructuredLoadAndSplit initialized successfully
INFO: Starting processing
INFO: Created initial document elements
INFO: Completed chunking process
INFO: Successfully obtained all documents


In [8]:
db.add(docs=docs)

INFO: Storing documents in the vector store
INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using engine: text-embedding-3-small


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


INFO: Document added successfully to the vector store.
INFO: Documents stored successfully


### Retrieve relevant information and generate an answer
The main purpose of these lines is to perform a query on the vector store to retrieve the most relevant information (top_k=5) and generate an answer using the language model.

In [11]:
query = "How Cinderella reach her happy ending?"
retriever = indox.RetrievalAugmentation(vector_database=db, llm=openai_qa, top_k=5)

invoke(query) method sends the query to the retriever, which searches the vector store for relevant text chunks and uses the language model to generate a response based on the retrieved information.
Context property retrieves the context or the detailed information that the retriever used to generate the answer to the query. It provides insight into how the query was answered by showing the relevant text chunks and any additional information used.

In [12]:
answer = retriever.invoke(query)
context = retriever.context

INFO: Retrieving context and scores from the vector database
INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using engine: text-embedding-3-small


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


INFO: Generating answer without document relevancy filter
INFO: Answering question
INFO: Generating response


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


INFO: Response generated successfully
INFO: Query answered successfully


In [13]:
answer

"Cinderella reached her happy ending by attending the three-day festival at the king's palace, where the prince instantly took her by the hand and danced only with her. Despite her step-sisters and step-mother not recognizing her in her beautiful attire, the prince identified her as the true bride when the golden slipper she left behind fit her perfectly. The two white doves confirmed her identity as the true bride, and Cinderella rode away with the prince on his horse. They were accompanied by the doves, symbolizing her purity and goodness. Ultimately, Cinderella's kindness, perseverance, and inner beauty led her to her happy ending of marrying the prince and living happily ever after."

### With AgenticRag

AgenticRag stands for Agentic Retrieval-Augmented Generation. This concept combines retrieval-based methods and generation-based methods in natural language processing (NLP). The key idea is to enhance the generative capabilities of a language model by incorporating relevant information retrieved from a database or a vector store. 
 AgenticRag is designed to provide more contextually rich and accurate responses by utilizing external knowledge sources. It retrieves relevant pieces of information (chunks) from a vector store based on a query and then uses a language model to generate a comprehensive response that incorporates this retrieved information.

In [ ]:
agent = indox.AgenticRag(llm=openai_qa, vector_database=db, top_k=5)
agent.run(query)